In [1]:
# 0 Housekeeping. Clear variable space
######################################
from IPython import get_ipython  

ipython = get_ipython()
ipython.magic("reset -f")
ipython = get_ipython()
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")

In [2]:
# 1 Import Libraries and Set Global Parameters
##############################################
# 1.1 Import Python Libraries
#############################
from datetime import datetime
print("Run Section 1 Import Libraries and Set Global Parameters...")
begin_time = datetime.now()

import os, sys
import pandas as pd
import geopandas as gpd
import pyarrow.parquet as pq

Run Section 1 Import Libraries and Set Global Parameters...


In [3]:
# 1.2 Set Global Parameters
###########################

if os.getlogin() == "WylieTimmerman":
    path_working = r"C:\OD\OneDrive - Foursquare ITP\Projects\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(r"C:\OD\OneDrive - Foursquare ITP\Projects\WMATA_AVL")
    path_sp = r"C:\Users\WylieTimmerman\Documents\projects_local\wmata_avl_local"
    path_source_data = os.path.join(path_sp,"data","00-raw")
    path_processed_data = os.path.join(path_sp, "data","02-processed")

elif os.getlogin() == "abibeka":
    path_working = r"C:\Users\abibeka\OneDrive - Kittelson & Associates, Inc\Documents\Github\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(path_working)
    path_source_data = r"C:\Users\abibeka\OneDrive - Kittelson & Associates, Inc\Documents\WMATA-AVL\Data"
    path_processed_data = os.path.join(path_source_data, "ProcessedData")
    
elif os.getlogin() == "E048374":
    # Working Paths
    path_working = r"C:\Users\E048374\OneDrive - WMATA\rawnav_rachel_fork\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(r"C:\Users\E048374\OneDrive - WMATA\rawnav_rachel_fork\WMATA_AVL")
    path_source_data = r"\\l-600730\RawNavArchive"
    path_sp = r"C:\Users\E048374\Documents\RawNav"
    path_processed_data = os.path.join(path_sp, "data", "02-processed")
    path_segments = path_processed_data

else:
    raise FileNotFoundError("Define the path_working, path_source_data, gtfs_dir, \
                            ZippedFilesloc, and path_processed_data in a new elif block")

In [4]:
# Globals
# Queue Jump Routes
q_jump_route_list = ['S4']
analysis_routes = q_jump_route_list
# EPSG code for WMATA-area work
wmata_crs = 2248
     
# 1.3 Import User-Defined Package
#################################
import wmatarawnav as wr

In [30]:
# 1.4 Reload Relevant Files 
###########################
# Load segment-pattern-stop crosswalk 
# This crosswalk is used to connect segment shapes to rawnav data. 
# - The 'route' field must match the same string used in the rawnav data. 
# - 'direction' will be looked up in a moment against the WMATA schedule database and 
#       replaced with a pattern code as an int32 value. 
# - 'seg_name_id' is also found in the segment geometry file. 
# - 'stop_id' matches the stop identifier in the WMATA schedule database.
xwalk_seg_pattern_stop_in = wr.tribble(
             ['route',        'direction',                        'seg_name_id','stop_id'], 
                 "79",            "SOUTH",               "georgia_columbia_stub",   10981, 
                 "79",            "SOUTH",           "georgia_piney_branch_stub",    4217, 
                 "70",            "SOUTH",                 "georgia_irving_stub",   19186, #irving stop
                 "70",            "SOUTH",               "georgia_columbia_stub",   10981, #columbia stop 
                 "70",            "SOUTH",           "georgia_piney_branch_stub",    4217,
                 "S1",            "NORTH",                    "sixteenth_u_stub",   18042,
                 "S2",            "NORTH",                    "sixteenth_u_stub",   18042,
                 "S4",            "NORTH",                    "sixteenth_u_stub",   18042,
                 "S9",            "NORTH",                    "sixteenth_u_stub",   18042,
                 "64",            "NORTH",            "eleventh_i_new_york_stub",   16490,
                 "G8",             "EAST",            "eleventh_i_new_york_stub",   16490,
                "D32",             "EAST",     "irving_fifteenth_sixteenth_stub",    2368,
                 "H1",            "NORTH",     "irving_fifteenth_sixteenth_stub",    2368,
                 "H2",             "EAST",     "irving_fifteenth_sixteenth_stub",    2368,
                 "H3",             "EAST",     "irving_fifteenth_sixteenth_stub",    2368,
                 "H4",             "EAST",     "irving_fifteenth_sixteenth_stub",    2368,
                 "H8",             "EAST",     "irving_fifteenth_sixteenth_stub",    2368,
                "W47",             "EAST",     "irving_fifteenth_sixteenth_stub",    2368
  )

xwalk_seg_pattern_stop_in.head(2)

,route,direction,seg_name_id,stop_id
0,79,SOUTH,georgia_columbia_stub,10981
1,79,SOUTH,georgia_piney_branch_stub,4217


In [7]:
wmata_schedule_dat = (
    pd.read_csv(
        os.path.join(path_sp, "wmata_schedule_data_q_jump_routes.csv"),
        index_col = 0
    )
    .reset_index(drop=True)
)

In [9]:
xwalk_wmata_route_dir_pattern = (wmata_schedule_dat
                                 .filter(items = ['direction', 'route','pattern'])
                                 .drop_duplicates()
                                )
xwalk_wmata_route_dir_pattern

,direction,route,pattern
0,SOUTH,52,1
42,NORTH,52,2
104,SOUTH,52,3
163,NORTH,52,4
208,SOUTH,S4,1
262,NORTH,S4,2


In [16]:
xwalk_seg_pattern_stop = (
    xwalk_seg_pattern_stop_in
    .merge(xwalk_wmata_route_dir_pattern, on = ['route','direction'])
    .drop('direction', 1)
    .reindex(columns = ['route','pattern','seg_name_id','stop_id'])
)
xwalk_seg_pattern_stop

,route,pattern,seg_name_id,stop_id
0,S4,2,sixteenth_u_stub,18042


In [12]:
# 2. Decompose Travel Time
##########################

# 2.0 Setup exports
###################
freeflow_list = []
stop_area_decomp_list = []
traveltime_decomp_list = []

path_exports = (
    os.path.join(
        path_processed_data,
        "exports_{}".format(datetime.now().strftime("%Y%m%d"))
    )
)
if not os.path.isdir(path_exports):
    os.mkdir(path_exports)

In [19]:
# 1 segment
seg = xwalk_seg_pattern_stop.loc[:,'seg_name_id'].values[0]
seg

'sixteenth_u_stub'

In [22]:
print('now on {}'.format(seg))
# 2.1. Read-in Data 
###################
# Reduce rawnav data to runs present in the summary file after filtering.

xwalk_seg_pattern_stop_fil = xwalk_seg_pattern_stop.query('seg_name_id == @seg')

seg_routes = list(xwalk_seg_pattern_stop_fil.route.drop_duplicates())

rawnav_dat = (
    wr.read_cleaned_rawnav(
       analysis_routes_ = seg_routes,
       path = os.path.join(path_processed_data, "rawnav_data.parquet")
    )
    .drop(columns=['blank', 'lat_raw', 'long_raw', 'sat_cnt'])
)

segment_summary = (
    pq.read_table(
        source = os.path.join(path_processed_data,"segment_summary.parquet"),
        filters = [['seg_name_id', "=", seg]],
        use_pandas_metadata = True
    )
    .to_pandas()
)

segment_summary_fil = (
    segment_summary
    .query('~(flag_too_far_any\
              | flag_wrong_order_any\
              | flag_too_long_odom\
              | flag_secs_total_mismatch\
              | flag_odom_total_mismatch)'
    )
)

now on sixteenth_u_stub


In [48]:
#first import stop_index -- then join seg_name_id
stop_index = ( pq.read_table(source=os.path.join(path_processed_data,"stop_index.parquet"),
                  filters=[[('route','=',route)] for route in seg_routes],
                    columns = [ 'route',
                                'pattern',
                                'stop_id',
                                'filename',
                                'index_run_start',
                                'index_loc',
                                'odom_ft',
                                'sec_past_st',
                                'geo_description'],
                  use_pandas_metadata = True
    )
    .to_pandas())

In [29]:
stop_index.head(3)

,route,pattern,stop_id,filename,index_run_start,index_loc,odom_ft,sec_past_st,geo_description
5616,S4,2,12189,rawnav04466200718.txt,21733.0,21776.0,71.0,318.0,10TH ST NW + CONSTITUTION AVE NW
5617,S4,2,1386,rawnav04466200718.txt,21733.0,21835.0,1169.0,439.0,12TH ST + CONSTITUTION AVE
5618,S4,2,17775,rawnav04466200718.txt,21733.0,21904.0,2483.0,562.0,11TH ST NW + E ST NW


In [36]:
# stop_index_w_segid = stop_index.merge(xwalk_seg_pattern_stop_in, how='left', on=['route','stop_id']).copy()

In [49]:
stop_index.loc[:,'pattern'] = stop_index.loc[:,'pattern'].astype('int32').copy()
stop_index.rename(columns={'odom_ft' : 'odom_ft_qj_stop'}, inplace=True)

In [51]:
# Filter Stop index to the relevant QJ stops
stop_index_fil = (
    stop_index
    .merge(xwalk_seg_pattern_stop_fil,
           on = ['route','pattern','stop_id'],
           how = 'inner')   
)

In [52]:
stop_index_fil

,route,pattern,stop_id,filename,index_run_start,index_loc,odom_ft_qj_stop,sec_past_st,geo_description,seg_name_id
0,S4,2,18042,rawnav04466200718.txt,21733.0,22410.0,13072.0,1867.0,16TH ST + U ST,sixteenth_u_stub
1,S4,2,18042,rawnav04466200718.txt,25450.0,26109.0,13056.0,1414.0,16TH ST + U ST,sixteenth_u_stub
2,S4,2,18042,rawnav04469200704.txt,19146.0,19621.0,9697.0,721.0,16TH ST + U ST,sixteenth_u_stub
3,S4,2,18042,rawnav04469200704.txt,22225.0,22820.0,12946.0,1085.0,16TH ST + U ST,sixteenth_u_stub
4,S4,2,18042,rawnav04469200704.txt,25736.0,26344.0,12937.0,1171.0,16TH ST + U ST,sixteenth_u_stub
...,...,...,...,...,...,...,...,...,...,...
764,S4,2,18042,rawnav07372200723.txt,6067.0,6584.0,13324.0,861.0,16TH ST + U ST,sixteenth_u_stub
765,S4,2,18042,rawnav07373200709.txt,8575.0,9099.0,12532.0,904.0,16TH ST + U ST,sixteenth_u_stub
766,S4,2,18042,rawnav08034200723.txt,6007.0,6555.0,12979.0,990.0,16TH ST + U ST,sixteenth_u_stub
767,S4,2,18042,rawnav08039200717.txt,5875.0,6367.0,12161.0,799.0,16TH ST + U ST,sixteenth_u_stub


In [57]:
segment_summary_fil.iloc[0]

filename                                                     rawnav04466200718.txt
index_run_start                                                              21733
pattern                                                                          2
start_date_time                                                2020-07-17 17:39:01
flag_too_far_any                                                             False
flag_wrong_order_any                                                         False
flag_too_long_odom                                                           False
flag_secs_total_mismatch                                                     False
flag_odom_total_mismatch                                                     False
start_odom_ft_segment                                                        12716
end_odom_ft_segment                                                          13422
trip_dist_mi_odom_and_segment                                                 0.13
star

In [42]:
# 2.2 Run Decomposition Functions
#################################
# These functions could be further nested within one another, but are kept separate to support
# easier review of intermediate outputs and to allow them to be used independently if needed.
# As a result, data is in some cases filtered several times, but the overall time loss is 
# small enough to be immaterial.

# Calculate Free Flow Travel Time through Entire Segment
segment_ff = (
    wr.decompose_segment_ff(
        rawnav_dat,
        segment_summary_fil,
        max_fps = 73.3
    )
    .assign(seg_name_id = seg)
)

freeflow_list.append(segment_ff)

In [43]:
freeflow_list

[      fps_next3        mph       seg_name_id
 0.01   0.425000   0.289707  sixteenth_u_stub
 0.05   1.642857   1.119875  sixteenth_u_stub
 0.10   6.500000   4.430811  sixteenth_u_stub
 0.15  12.000000   8.179959  sixteenth_u_stub
 0.25  19.000000  12.951602  sixteenth_u_stub
 0.50  30.000000  20.449898  sixteenth_u_stub
 0.75  38.666667  26.357646  sixteenth_u_stub
 0.85  42.666667  29.084299  sixteenth_u_stub
 0.90  45.333333  30.902068  sixteenth_u_stub
 0.95  49.000000  33.401500  sixteenth_u_stub
 0.99  58.666667  39.990911  sixteenth_u_stub]

In [44]:
# Calculate Stop-Area Decomposition
stop_area_decomp = (
    wr.decompose_stop_area(
        rawnav_dat,
        segment_summary_fil,
        stop_index_fil
    )
    .assign(seg_name_id = seg)
)

stop_area_decomp_list.append(stop_area_decomp)

In [61]:
stop_area_decomp_list[0].iloc[0]

index                                          13
index_loc                                   22404
lat                                       38.9164
long                                     -77.0365
heading                                       360
door_state                                      C
veh_state                                       M
odom_ft                                     12947
sec_past_st                                  1862
stop_window                                      
row_before_apc                                  0
route_pattern                                S402
pattern                                         2
index_run_start                             21733
index_run_end                               23696
filename                    rawnav04466200718.txt
start_date_time               2020-07-17 17:39:01
route                                          S4
wday                                       Friday
odom_ft_next                                12947


In [58]:
segment_ff_val = (
    segment_ff
    .loc[0.95]
    .loc["fps_next3"]
)

# Run decomposition
traveltime_decomp = (
    wr.decompose_traveltime(
        rawnav_dat,
        segment_summary_fil,
        stop_area_decomp,
        segment_ff_val
    )
)
traveltime_decomp_list.append(traveltime_decomp)

In [62]:
traveltime_decomp_list[0].iloc[0]

filename             rawnav04466200718.txt
index_run_start                      21733
seg_name_id               sixteenth_u_stub
route                                   S4
pattern                                  2
                             ...          
lat_start                          38.8928
long_start                        -77.0261
lat_end                            38.9943
long_end                           -77.031
odom_ft_seg_total                      706
Name: 0, Length: 61, dtype: object

In [63]:
# 3. Export
###########

# 3.1 Combine results by segment into tables   
########################################
freeflow = (
    pd.concat(freeflow_list)
    .rename_axis('ntile')
    .reset_index()
)

basic_decomp = (
    pd.concat(stop_area_decomp_list)
    .reset_index() 
)

traveltime_decomp = (
    pd.concat(traveltime_decomp_list)
    .reset_index()
)

In [64]:
# 3.2 Export Values
###################

freeflow.to_csv(os.path.join(path_exports,"freeflow.csv"))

basic_decomp.to_csv(os.path.join(path_exports,"basic_decomp.csv"))

traveltime_decomp.to_csv(os.path.join(path_exports,"traveltime_decomp.csv"))